In [97]:
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Dropout
from keras.models import Model, Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
import keras.utils as image
import matplotlib.pyplot as plt

# Set the input shape and number of classes
INPUT_SHAPE = (128, 128, 3)  # Assume 128x128 RGB images
NUM_CLASSES = 2  # Food or not food

# Set the batch size
BATCH_SIZE = 64

# Set the directory containing the images
image_dir = 'food_images_2'

# Create an ImageDataGenerator object with the preprocessing function
image_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Use the generator to load the images from the directory
train_data_gen = image_generator.flow_from_directory(image_dir,
                                                     target_size=(128, 128),
                                                     color_mode='rgb',
                                                     class_mode='binary',
                                                     batch_size=32,
                                                     subset='training')
val_data_gen = image_generator.flow_from_directory(image_dir,
                                                   target_size=(128, 128),
                                                   color_mode='rgb',
                                                   class_mode='binary',
                                                   batch_size=32,
                                                   subset='validation')

# Get the images and labels from the generator
x_train, y_train = next(train_data_gen)
x_val, y_val = next(val_data_gen)


# Define the encoder
encoder_input = layers.Input(shape=INPUT_SHAPE)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(encoder_input)
x = layers.MaxPooling2D(2, padding='same')(x)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2, padding='same')(x)
x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2, padding='same')(x)
x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2, padding='same')(x)
x = layers.Dropout(0.5)(x)  # Add dropout for regularization
encoder_output = layers.MaxPooling2D(2, padding='same')(x)

# Create the encoder model
encoder = Model(encoder_input, encoder_output, name='encoder')

# Define the decoder
decoder_input = encoder_output
x = layers.Conv2D(256, 3, activation='relu', padding='same')(decoder_input)
x = layers.UpSampling2D(2)(x)
x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
x = layers.UpSampling2D(2)(x)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.UpSampling2D(2)(x)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = layers.UpSampling2D(2)(x)
x = layers.UpSampling2D(2)(x)
decoder_output = layers.Conv2D(3, 3, activation='softmax', padding='same')(x)

# Print encoder and decoder shapes
print(encoder_input.shape)
print(encoder_output.shape)
print(decoder_input.shape)
print(decoder_output.shape)


Found 800 images belonging to 1 classes.
Found 199 images belonging to 1 classes.
(None, 128, 128, 3)
(None, 4, 4, 256)
(None, 4, 4, 256)
(None, 128, 128, 3)


In [93]:
# Define the model
autoencoder = Model([encoder_input, decoder_input], decoder_output)

# Compile the model
autoencoder.compile(loss='binary_crossentropy', 
                    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))


ValueError: Found unvisited input tensors that are disconnected from the outputs: [<KerasTensor: shape=(None, 128, 128, 3) dtype=float32 (created by layer 'input_79')>]

In [89]:
history = autoencoder.fit([x_train, x_train], y_train,
                          batch_size=BATCH_SIZE,
                          epochs=10,
                          validation_data=([x_val, x_val], y_val))


Epoch 1/10


ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb

In [78]:
# Define the encoder model
encoder_model = Model(encoder_input, encoder_output)

# Define the decoder model
decoder_model = Model(decoder_input, decoder_output)

In [79]:
# Train the encoder and decoder separately
history_encoder = encoder_model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=10, validation_data=(x_val, y_val), verbose=2)
history_decoder = decoder_model.fit(y_train, x_train, batch_size=BATCH_SIZE, epochs=10, validation_data=(y_val, x_val), verbose=2)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [ ]:
# Save the weights of the autoencoder
autoencoder.save_weights('autoencoder_weights.h5')

# Use the encoder to generate a lower-dimensional representation of the input images
X_encoded = encoder.predict(x_train)

# Train a classifier on the lower-dimensional representation of the input images
classifier = Sequential()
classifier.add(Flatten(input_shape=X_encoded.shape[1:]))  # Flatten the input
classifier.add(Dense(64, activation='relu'))
classifier.add(Dropout(0.5))  # Add dropout for regularization
classifier.add(Dense(32, activation='relu'))
classifier.add(Dropout(0.5))  # Add dropout for regularization
classifier.add(Dense(1, activation='sigmoid'))
classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.fit(X_encoded, y_train, epochs=10, batch_size=32)

# Save the classifier
classifier.save('classifier.h5')

In [ ]:
# Save the weights of the autoencoder
autoencoder.save_weights('autoencoder_weights.h5')

# Use the encoder to generate a lower-dimensional representation of the input images
X_encoded = encoder.predict(x_train)

# Train a classifier on the lower-dimensional representation of the input images
classifier = Sequential()
classifier.add(Flatten(input_shape=X_encoded.shape[1:]))  # Flatten the input
classifier.add(Dense(64, activation='relu'))
classifier.add(Dropout(0.5))  # Add dropout for regularization
classifier.add(Dense(32, activation='relu'))
classifier.add(Dropout(0.5))  # Add dropout for regularization
classifier.add(Dense(1, activation='sigmoid'))
classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.fit(X_encoded, y_train, epochs=10, batch_size=32)

# Save the classifier
classifier.save('classifier.h5')

In [31]:
import imageio
import numpy as np
from skimage.transform import resize

# Load the autoencoder and the classifier
#autoencoder = load_model('autoencoder.h5')
#classifier = load_model('classifier.h5')

# Load the image and preprocess it
image_data = imageio.imread('./tripadvisor_images_2/23487213_1.jpg')
# Resize the image to the same size as the training images
resized_image = resize(image_data, (128, 128))
# Normalize the image data
resized_image /= 255.
# Add an extra dimension to the array to match the expected input shape of the model
resized_image = np.expand_dims(resized_image, axis=0)


# Use the autoencoder to generate a lower-dimensional representation of the input image
X_encoded = autoencoder.predict(resized_image)

# Use the classifier to make a prediction on the lower-dimensional representation of the input image
prediction = classifier.predict(X_encoded)[0][0]

if prediction > 0.5:
    print('Food image')
else:
    print('Not a food image')


1/1 [==============================] - 0s 27ms/step


/var/folders/4l/gsvsrpxd2kqb4f6v1wy203b80000gn/T/ipykernel_8046/1061904928.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image_data = imageio.imread('./tripadvisor_images_2/23487213_1.jpg')


ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 2079, in predict_step
        return self(x, training=False)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filter